In [1]:
import spacy
import pandas as pd
import matplotlib.pyplot as mltp
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import sklearn as sk
import seaborn as sea
import re 
from tqdm import tqdm

In [2]:
from transformers import pipeline

In [1]:
classifier = pipeline("zero-shot-classification")

NameError: name 'pipeline' is not defined

In [4]:
labels=['climate',
'economy',
'education',
'health',
'infrastructure',
'science',
'social causes',
'politics and ideology',
'technology and entrepreneurship']

In [2]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [3]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
nli_model.to(mps_device)
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')



In [5]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
premise = sequence_to_classify
hypothesis = f'This example is {candidate_labels}.'

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                     truncation_strategy='only_first')
logits = nli_model(x.to(mps_device))[0]

/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/torch/functional.py:864: UserWarning: The operator 'aten::unique_consecutive' is not currently sup

In [11]:
logits

tensor([[ 0.2638,  0.6775, -1.0134]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [12]:

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]

In [17]:
prob_label_is_true

tensor([0.5418], device='mps:0', grad_fn=<SelectBackward0>)

In [14]:
class_bcandidate_labels = ["positive", "negative",'neutral']

In [6]:
sequence = "I hated this movie. The acting sucked."
classifier(sequence, class_bcandidate_labels)

{'sequence': 'I hated this movie. The acting sucked.',
 'labels': ['negative', 'neutral', 'positive'],
 'scores': [0.9757519960403442, 0.016008835285902023, 0.008239180780947208]}

In [7]:
sequence = "The movie was 100 minutes long, with no pause."
classifier(sequence, class_bcandidate_labels)

{'sequence': 'The movie was 100 minutes long, with no pause.',
 'labels': ['negative', 'positive', 'neutral'],
 'scores': [0.3668157756328583, 0.34670284390449524, 0.28648144006729126]}

In [8]:
sequence = "Die Wirtschaft verlief gut"


classifier(sequence, labels, multi_class=True)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


{'sequence': 'Die Wirtschaft verlief gut',
 'labels': ['economy',
  'infrastructure',
  'technology and entrepreneurship',
  'health',
  'science',
  'social causes',
  'climate',
  'education',
  'politics and ideology'],
 'scores': [0.9657909870147705,
  0.13397930562496185,
  0.024615313857793808,
  0.0004468511324375868,
  0.0003251165908295661,
  0.000202704148250632,
  0.00012562797928694636,
  8.024831186048687e-05,
  2.9401642677839845e-05]}

In [9]:
classifier_2 = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [10]:
sequence = "Die Wirtschaft verlief gut"


classifier_2(sequence, labels, multi_class=True)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


{'sequence': 'Die Wirtschaft verlief gut',
 'labels': ['economy',
  'infrastructure',
  'technology and entrepreneurship',
  'health',
  'science',
  'social causes',
  'climate',
  'education',
  'politics and ideology'],
 'scores': [0.9657909870147705,
  0.13397930562496185,
  0.024615313857793808,
  0.0004468511324375868,
  0.0003251165908295661,
  0.000202704148250632,
  0.00012562797928694636,
  8.024831186048687e-05,
  2.9401642677839845e-05]}

In [ ]:
def lemmatization_test(texts,  allowed_posttags=['NOUN','ADJ','VERB','ADV']):
    nlp=spacy.load('en_core_web_sm',disable=['parser','ner'])
    texts_out=[]
    for text in tqdm(texts):
        doc= nlp(text)
        new_text=[]
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final=' '.join(new_text)
        texts_out.append(final)
    return texts_out